# Actividad de aprendizaje 4: Programación por objetos

For this case, I used some standars to practice good pracitces:

PEP 8 (identation), PEP 257 (DOCSTRINGS), PEP 318 (DECORATOR), PEP 454 (TYPE HINTS)

## 1. Estás desarrollando un sistema para gestionar una biblioteca digital. Este sistema debe permitir a los usuarios añadir libros, buscar libros por diferentes criterios, tomar prestado libros y gestionar devoluciones.


To make this digiital library work, I would need to create a class for the library, and then create a class for the books.
The library class would have a list of books, and methods to add, remove, lookup, borrow and return books from the list.
The book class would have attributes for title, author, and availability, and method to change the availability of the book.

I would also need to create:
1.    A menu for the user to interact with the library, and a loop to keep the program running until the user decides to exit.
2.    File to store the books in the library, so that the library can be reloaded with the same books when the program is restarted.
3.    A way to save the changes to the library to the file when the program is exited.

In [1]:
from typing import Any, Dict, List, Optional


class Book:
    def __init__(self, *args: Any, **kwargs: Dict[str, Any]) -> None:
        """
        Initialize a new book instance.

        Args:
            title (str): The title of the book.
            author (str): The author of the book.
            year (str): The year the book was published.
            genre (str): The genre of the book.
        """
        self.title = kwargs.get('title', 'Unknown Title')
        self.author = kwargs.get('author', 'Unknown Author')
        self.year = kwargs.get('year', 'Unknown Year')
        self.genre = kwargs.get('genre', 'Unknown Genre')
        self.availability = True

    @property
    def info(self) -> str:
        """
        Return a string representation of the book.

        Returns:
            str: A string representing the book.
        """
        return f'{self.title} by {self.author} ({self.year})'
    
    @property
    def availability(self) -> bool:
        """Get the availability of the book."""
        return self._availability

    @availability.setter
    def availability(self, value: bool) -> None:
        """Set the availability of the book."""
        
        if not isinstance(value, bool):
            raise ValueError("Availability must be a boolean.")
        self._availability = value

    def borrow_book(self) -> bool:
        """
        Mark the book as borrowed.

        Returns:
            bool: True if the book was successfully borrowed, False otherwise.
        """
        try:
            self.availability = False
            return True
        except:
            raise Exception('Book is not available')
        
    def return_book(self) -> bool:
        """
        Mark the book as returned.

        :return: True if the book was successfully returned, False otherwise.
        """
        try:
            self.availability = True
            return True
        except:
            raise Exception('Book is already available')

In [2]:
import sqlite3

class DigitalLibrary:
    def __init__(self, db_name: str ="library.db") -> None:
        self.conn = sqlite3.connect(db_name)
        self.create_table()

    def create_table(self) -> None:
        """Create the books table if it doesn't exist."""
        with self.conn:
            self.conn.execute('''
                CREATE TABLE IF NOT EXISTS books (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT NOT NULL,
                    author TEXT NOT NULL,
                    year TEXT NOT NULL,
                    genre TEXT NOT NULL,
                    availability INTEGER NOT NULL
                )
            ''')

    def add_book(self, book: Book) -> None:
        """Add a new book to the library."""
        with self.conn:
            self.conn.execute('''
                INSERT INTO books (title, author, year, genre, availability)
                VALUES (?, ?, ?, ?, ?)
            ''', (book.title, book.author, book.year, book.genre, int(book.availability)))

    def lookup_book(self, title: str) -> Optional[List]:
        """Find a book by title."""
        cursor = self.conn.cursor()
        cursor.execute('SELECT * FROM books WHERE title = ?', (title,))
        return cursor.fetchone()

    def borrow_book(self, title: str) -> bool:
        """Mark a book as borrowed."""
        with self.conn:
            cursor = self.conn.cursor()
            cursor.execute('SELECT availability FROM books WHERE title = ?', (title,))
            book = cursor.fetchone()
            if book and book[0] == 1:  # Check if book is available
                cursor.execute('UPDATE books SET availability = 0 WHERE title = ?', (title,))
                return True
        return False

    def return_book(self, title: str) -> bool:
        """Mark a book as returned."""
        with self.conn:
            cursor = self.conn.cursor()
            cursor.execute('SELECT availability FROM books WHERE title = ?', (title,))
            book = cursor.fetchone()
            if book and book[0] == 0:  # Check if book is borrowed
                cursor.execute('UPDATE books SET availability = 1 WHERE title = ?', (title,))
                return True
        return False

    def save_library(self) -> None:
        """Commit changes to the database."""
        self.conn.commit()

    def load_library(self) -> None:
        """Load library from the database (not necessary since we query directly)."""
        pass  # All interactions are done directly with the database

    def display_books(self) -> None:
        """Display all books in the library."""
        cursor = self.conn.cursor()
        cursor.execute('SELECT title, author, availability FROM books')
        books = cursor.fetchall()
        for book in books:
            print(f"{book[0]} by {book[1]} - {'Available' if book[2] == 1 else 'Borrowed'}")

    def main_menu(self) -> None:
        """Main menu for library interaction."""
        def show_menu():
            print("1. Add Book")
            print("2. Lookup Book")
            print("3. Borrow Book")
            print("4. Return Book")
            print("5. Display Books")
            print("6. Exit")
        
        show_menu()

        while True:
            choice = input("\nEnter your choice: ")

            if choice == '1':
                title = input("Enter title: ")
                author = input("Enter author: ")
                year = input("Enter year: ")
                genre = input("Enter genre: ")
                book = Book(title=title, author=author, year=year, genre=genre)
                self.add_book(book)
            elif choice == '2':
                title = input("Enter title: ")
                book = self.lookup_book(title)
                if book:
                    print(f"Book found: {book[1]} by {book[2]} - {'Available' if book[5] else 'Borrowed'}")
                else:
                    print("Book not found.")
            elif choice == '3':
                title = input("Enter title: ")
                if self.borrow_book(title):
                    print("Book borrowed successfully.")
                else:
                    print("Book could not be borrowed.")
            elif choice == '4':
                title = input("Enter title: ")
                if self.return_book(title):
                    print("Book returned successfully.")
                else:
                    print("Book could not be returned.")
            elif choice == '5':
                self.display_books()
            elif choice == '6':
                break
            else:
                print("Invalid choice")

    def __del__(self) -> None:
        """Close the database connection when the library is destroyed."""
        self.conn.close()


In [3]:
def add_books(library):
    # Create and add 5 books to the library using the Book class
    books = [
        Book(title="To Kill a Mockingbird", author="Harper Lee", year="1960", genre="Fiction"),
        Book(title="1984", author="George Orwell", year="1949", genre="Dystopian"),
        Book(title="The Great Gatsby", author="F. Scott Fitzgerald", year="1925", genre="Fiction"),
        Book(title="The Catcher in the Rye", author="J.D. Salinger", year="1951", genre="Fiction"),
        Book(title="Moby-Dick", author="Herman Melville", year="1851", genre="Adventure"),
    ]

    for book in books:
        library.add_book(book)


if __name__ == "__main__":
    TecLibrary = DigitalLibrary()

    if TecLibrary.get_book_count() < 5:
        add_books(TecLibrary)
    
    TecLibrary.main_menu()

1. Add Book
2. Lookup Book
3. Borrow Book
4. Return Book
5. Display Books
6. Exit
Invalid choice
El Principito by Desconocido - Borrowed
Book found: El Principito by Desconocido - Borrowed
Book returned successfully.
El Principito by Desconocido - Available
El Principito by Desconocido - Available
Web by Desconocido - Available


## 2.  Estás desarrollando un sistema para gestionar las reservas de mesas en un restaurante. El sistema debe permitir añadir reservas, gestionar el estado de las mesas, buscar reservas por cliente o por fecha, y generar informes sobre el uso de las mesas.